In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("house_price_bd.csv")

removing duplicates

In [17]:
df.drop_duplicates(inplace = True)

In [18]:
df.shape

(2931, 9)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2931 entries, 0 to 3862
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             2931 non-null   object 
 1   Bedrooms          2100 non-null   float64
 2   Bathrooms         2100 non-null   float64
 3   Floor_no          2356 non-null   object 
 4   Occupancy_status  2842 non-null   object 
 5   Floor_area        2842 non-null   float64
 6   City              2931 non-null   object 
 7   Price_in_taka     2931 non-null   object 
 8   Location          2925 non-null   object 
dtypes: float64(3), object(6)
memory usage: 229.0+ KB


dropping rows with no locations

In [20]:
df=df.dropna(subset=['Location'])

In [21]:
df['City'].value_counts()

City
chattogram          1162
dhaka               1090
narayanganj-city     367
gazipur              169
cumilla              137
Name: count, dtype: int64

keeping rows of 'dhaka' only 

In [22]:
df=df[df['City']=='dhaka']

seperating 'Area' & 'Section' from 'Location'

In [23]:
split_df = df['Location'].str.split(',', expand=True)
df['Area'] = split_df[1].where(df['Location'].str.count(',')==1).str.strip().str.lower()
df['Section'] = split_df[0].where(df['Location'].str.count(',')==1).str.strip().str.lower().str.replace('sector', 'section')

In [24]:
df.head()

,Title,Bedrooms,Bathrooms,Floor_no,Occupancy_status,Floor_area,City,Price_in_taka,Location,Area,Section
0,We Are Offering You A Very Spacious 1960 Sq Ft...,3.0,4.0,3,vacant,1960.0,dhaka,"৳39,000,000","Gulshan 1, Gulshan",gulshan,gulshan 1
1,Valuable 1705 Square Feet Apartment Is Ready T...,3.0,3.0,1,vacant,1705.0,dhaka,"৳16,900,000","Lake Circus Road, Kalabagan",kalabagan,lake circus road
2,1370 square feet apartment is ready to sale in...,3.0,3.0,6,vacant,1370.0,dhaka,"৳12,500,000","Shukrabad, Dhanmondi",dhanmondi,shukrabad
3,2125 Square Feet Apartment For Sale In Bashund...,3.0,3.0,4,vacant,2125.0,dhaka,"৳20,000,000","Block L, Bashundhara R-A",bashundhara r-a,block l
4,Buy This 2687 Square Feet Flat In The Nice Are...,3.0,3.0,4,vacant,2687.0,dhaka,"৳47,500,000","Road No 25, Banani",banani,road no 25


keeping only rows of 'mirpur' in area

In [28]:
df=df[df['Area']=='mirpur']

In [27]:
df[df['Area']=='mirpur']['Section'].value_counts()

Section
section 10                  33
section 12                  31
section 1                   28
ahmed nagar                 25
pallabi                     24
pirerbag                    24
west shewrapara             23
section 11                  22
middle monipur              21
section 6                   20
east kazipara               17
section 2                   16
mirpur dohs                 15
kallyanpur                  14
rupnagar r/a                12
darussalam                  10
east monipur                 8
paikpara                     8
west monipur                 8
middle paikpara              8
dakkhin paikpara             6
section 15                   6
2nd colony                   5
east shewrapara              5
west kazipara                5
1st colony                   4
south monipur                4
section 7                    4
arambag residential area     2
section 13                   1
lalkuthi                     1
kachukhet road               1


handling missing values

In [29]:
df.isna().sum()

Title                0
Bedrooms            14
Bathrooms           14
Floor_no            13
Occupancy_status     5
Floor_area           5
City                 0
Price_in_taka        0
Location             0
Area                 0
Section              0
dtype: int64

In [34]:
df["Floor_no"].value_counts()

Floor_no
1     59
5     59
4     56
3     51
2     50
6     50
7     29
8     25
9     15
13     2
11     2
Name: count, dtype: int64

In [32]:
df[df["Floor_no"]=='G+7']

,Title,Bedrooms,Bathrooms,Floor_no,Occupancy_status,Floor_area,City,Price_in_taka,Location,Area,Section
431,19200 SQ FT Full-Building is now for sale in M...,46.0,10.0,G+7,vacant,19200.0,dhaka,"৳75,000,000","Section 1, Mirpur",mirpur,section 1


it is a full building that's why "G+7" means that it is a 8 floor building.hence replacing with 8.

In [33]:
df["Floor_no"] = df["Floor_no"].str.replace(r"G\+7|0\+7", "8", regex=True)

creating new column property type

In [35]:
building_pattern = r"building|complex|condominium|residence"
apartment_pattern = r"apartment|flat|living space|apt.|duplex"
commercial_pattern = r"office|shop|retail|business|commercial space|warehouse|factory|store|business center|showroom|industrial space|market|plaza|outlet|commercial"
house_pattern = r"house|residential|cottage|bungalow|townhouse|family home|detached house|semi-detached house|ranch|estate|home"

In [37]:
import re



# Create the 'property_type' column
df['property_type'] = 'Unknown'

# Apply patterns to the 'title' column
df.loc[df['Title'].str.contains(building_pattern, flags=re.IGNORECASE, regex=True), 'property_type'] = 'building'
df.loc[df['Title'].str.contains(apartment_pattern, flags=re.IGNORECASE, regex=True), 'property_type'] = 'apartment'
df.loc[df['Title'].str.contains(commercial_pattern, flags=re.IGNORECASE, regex=True), 'property_type'] = 'commercial'
df.loc[df['Title'].str.contains(house_pattern, flags=re.IGNORECASE, regex=True), 'property_type'] = 'bungalow/townhouse'


In [48]:
df[df['property_type']=='Unknown']

,Title,Bedrooms,Bathrooms,Floor_no,Occupancy_status,Floor_area,City,Price_in_taka,Location,Area,Section,property_type


we can see that there are lot of land properties so we can remove those since we are only working with housing

In [44]:
# prompt: remove rows where the Title contains ' Plot| plot| Katha| katha'

import pandas as pd
import re

# Assuming your DataFrame is named 'df_clean'
# Create a copy to avoid modifying the original DataFrame directly


# Define the pattern for the strings to remove
pattern = r"plot| katha"

# Use boolean indexing to filter out rows where 'Title' matches the pattern
df = df[~df['Title'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]


In [47]:
# prompt: for the rows that have 'Unknown' in the property_type column replace those with 'apartment' on property_type

# Replace 'Unknown' values in 'property_type' with 'apartment'
df.loc[df['property_type'] == 'Unknown', 'property_type'] = 'apartment'


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 408 entries, 25 to 1641
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             408 non-null    object 
 1   Bedrooms          396 non-null    float64
 2   Bathrooms         396 non-null    float64
 3   Floor_no          398 non-null    object 
 4   Occupancy_status  403 non-null    object 
 5   Floor_area        403 non-null    float64
 6   City              408 non-null    object 
 7   Price_in_taka     408 non-null    object 
 8   Location          408 non-null    object 
 9   Area              408 non-null    object 
 10  Section           408 non-null    object 
 11  property_type     408 non-null    object 
dtypes: float64(3), object(9)
memory usage: 41.4+ KB


replacing the missing bedrooms,bathrooms and Floor_no values with median values of the individual property_type.

In [53]:
df["Floor_no"]=df["Floor_no"].astype(float)

In [54]:
# Find rows where 'property_type' is 'building' and 'Bedrooms' or 'Bathrooms' are missing.
missing_bedrooms_bathrooms_buildings = df[
    (df['property_type'] == 'building') &
    (df['Bedrooms'].isna() | df['Bathrooms'].isna())
]
missing_bedrooms_bathrooms_apartments = df[
    (df['property_type'] == 'apartment') &
    (df['Bedrooms'].isna() | df['Bathrooms'].isna())
]
missing_bedrooms_bathrooms_commercial = df[
    (df['property_type'] == 'commercial') &
    (df['Bedrooms'].isna() | df['Bathrooms'].isna())
]
missing_bedrooms_bathrooms_bungalow = df[
    (df['property_type'] == 'bungalow/townhouse') &
    (df['Bedrooms'].isna() | df['Bathrooms'].isna())
]


num_rows_missing_building = len(missing_bedrooms_bathrooms_buildings)
num_rows_missing_commercial = len(missing_bedrooms_bathrooms_commercial)
num_rows_missing_apartment = len(missing_bedrooms_bathrooms_apartments)
num_rows_missing_bungalow = len(missing_bedrooms_bathrooms_bungalow)
print(f"Number of rows with 'building' property type and missing Bedrooms or Bathrooms: {num_rows_missing_building}")
print(f"Number of rows with 'commercial' property type and missing Bedrooms or Bathrooms: {num_rows_missing_commercial}")
print(f"Number of rows with 'apartment' property type and missing Bedrooms or Bathrooms: {num_rows_missing_apartment}")
print(f"Number of rows with 'bungalow' property type and missing Bedrooms or Bathrooms: {num_rows_missing_bungalow}")



Number of rows with 'building' property type and missing Bedrooms or Bathrooms: 0
Number of rows with 'commercial' property type and missing Bedrooms or Bathrooms: 7
Number of rows with 'apartment' property type and missing Bedrooms or Bathrooms: 3
Number of rows with 'bungalow' property type and missing Bedrooms or Bathrooms: 2


In [55]:
df_clean=df.copy()
# Create dictionaries to hold median floor numbers
median_bathroom_no = {
    "building": None,
    "apartment": None,
    "bungalow/townhouse": None,
    "commercial":None
}

# Calculate median floor_no for each property type
for property_type in ["building", "apartment", "bungalow/townhouse", "commercial"]:
    sub_df = df_clean[df_clean['property_type'] == property_type]
    median_bathroom_no[property_type] = sub_df["Bathrooms"].median()
    # Create dictionaries to hold median floor numbers
median_bedroom_no = {
    "building": None,
    "apartment": None,
    "bungalow/townhouse": None
}

# Calculate median floor_no for each property type
for property_type in ["building", "apartment", "bungalow/townhouse", "commercial"]:
    sub_df = df_clean[df_clean['property_type'] == property_type]
    median_bedroom_no[property_type] = sub_df["Bedrooms"].median()
# Create dictionaries to hold median floor numbers
median_floor_no = {
    "building": None,
    "apartment": None,
    "bungalow/townhouse": None,
    "commercial": None
}

# Calculate median floor_no for each property type
for property_type in ["building", "apartment", "bungalow/townhouse",'commercial']:
    sub_df = df_clean[df_clean['property_type'] == property_type]
    median_floor_no[property_type] = sub_df["Floor_no"].median()
# Fill missing 'Floor_no' based on the median for each property type
for property_type in median_floor_no:
    df_clean.loc[(df_clean['Floor_no'].isna()) & (df_clean['property_type'] == property_type), 'Floor_no'] = median_floor_no[property_type]

# Fill missing 'Bathrooms' based on the median for each property type
for property_type in median_bathroom_no:

    df_clean.loc[(df_clean['Bathrooms'].isna()) & (df_clean['property_type'] == property_type), 'Bathrooms'] = median_bathroom_no[property_type]

# Fill missing 'Bedrooms' based on the median for each property type
for property_type in median_bedroom_no:
    df_clean.loc[(df_clean['Bedrooms'].isna()) & (df_clean['property_type'] == property_type), 'Bedrooms'] = median_bedroom_no[property_type]



taking the missing Floor_area from title

In [57]:
df_copy=df_clean.copy()
import re

def extract_floor_area(title):
  """
  Extracts floor area from a title string.

  Args:
    title: The title string to search.

  Returns:
    The extracted floor area as a float, or None if not found.
  """
  match = re.search(r"(\d+)\s*sq", title.lower())
  if match:
    return float(match.group(1))
  return None


# Example usage (assuming df_clean is your DataFrame)

df_copy.loc[df_copy['Floor_area'].isna(), 'Floor_area'] = df_copy.loc[df_copy['Floor_area'].isna(), 'Title'].apply(extract_floor_area)
  #df_copy.loc[i, 'Floor_area'] = extract_floor_area(df_copy.loc[i, 'Title'])
#df_clean['extracted_floor_area'] = df_clean['Title'].apply(extract_floor_area)

# Now you can use the 'extracted_floor_area' column to fill missing values in 'Floor_area'
# Example: Fill missing values in 'Floor_area' with values from 'extracted_floor_area'



filling the missing Occupancy_status to vacant

In [59]:
df_copy.loc[df_copy['Occupancy_status'].isna(),'Occupancy_status']='vacant'

In [61]:
df_copy.isna().sum()

Title               0
Bedrooms            0
Bathrooms           0
Floor_no            0
Occupancy_status    0
Floor_area          0
City                0
Price_in_taka       0
Location            0
Area                0
Section             0
property_type       0
dtype: int64

our dataset is ready! But we have to convert Price_in_taka to float

In [64]:
df_copy['Price_in_taka']=df_copy['Price_in_taka'].replace({'\৳': '', ',': ''}, regex=True).astype(float)

In [65]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 408 entries, 25 to 1641
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             408 non-null    object 
 1   Bedrooms          408 non-null    float64
 2   Bathrooms         408 non-null    float64
 3   Floor_no          408 non-null    float64
 4   Occupancy_status  408 non-null    object 
 5   Floor_area        408 non-null    float64
 6   City              408 non-null    object 
 7   Price_in_taka     408 non-null    float64
 8   Location          408 non-null    object 
 9   Area              408 non-null    object 
 10  Section           408 non-null    object 
 11  property_type     408 non-null    object 
dtypes: float64(5), object(7)
memory usage: 41.4+ KB


In [66]:
df_copy.to_csv("housing_cleaned.csv",index=False)